## Generation of UIDB

In [1]:
import pandas as pd
import numpy as np
import os
import re
import gc
from pathlib import Path
import itertools

# ==============================================================================
# 1. CONFIGURACIÓN GLOBAL Y RUTAS
# ==============================================================================

# Ruta Única de Salida
OUTPUT_DIR = Path(r"C:\Users\Eydan\OneDrive\Escritorio\ITESM\MAITEC Lab\VISUM_MOVES Integration\NBs for MOVES VISUM\NB2_MOVES UIDB from VISUM\NB2_OutputData\MOVES_UIDB_Tables_VISUM\Complete Tables")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Parámetros Globales MOVES
YEAR_ID = 2024
DAY_IDS = [5, 2]
MILES_PER_KM = 0.621371
MOVES_SPEED_BINS = [0, 2.5, 7.5, 12.5, 17.5, 22.5, 27.5, 32.5, 37.5, 42.5, 47.5, 52.5, 57.5, 62.5, 67.5, 72.5, float('inf')]

HOURS_MAPPING = {
    "EM": [22, 23, 0, 1, 2, 3, 4, 5], "AP": [6, 7, 8, 9], "LM": [10, 11],
    "MP": [12], "EA": [13, 14, 15, 16], "PP": [17, 18], "EV": [19, 20, 21]
}

ALL_ROAD_TYPES = [1, 2, 3, 4, 5]
ALL_HOURS = range(1, 25)
ALL_BINS = range(1, 17)

# ==============================================================================
# 2. CONFIGURACIÓN AUTO Y CARGA
# ==============================================================================

INPUT_VISUM_DIR_AUTO = Path(r"C:\Users\Eydan\OneDrive\Escritorio\ITESM\MAITEC Lab\VISUM_MOVES Integration\NBs for MOVES VISUM\NB2_MOVES UIDB from VISUM\NB2_IntputData\Links Auto")

FILES_MAPPING_AUTO = {
    "EM": "scenario=AutoEM", "AP": "scenario=AutoAP", "LM": "scenario=AutoLM",
    "MP": "scenario=AutoMP", "EA": "scenario=AutoEA", "PP": "scenario=AutoPP", "EV": "scenario=AutoEV"
}

VOLUME_FACTORS = {
    "EM": 1.0, "AP": 0.290781, "LM": 0.503228, "MP": 1.0, "EA": 0.250912, "PP": 0.512564, "EV": 0.406226 
}

AUTO_LIGHT_MAPPING = {
    11: 0.146429251, 21: 0.320888794, 22: 0.104594992,
    31: 0.190505261, 32: 0.220278756, 43: 0.002389634, 44: 0.014913313
}

KM_DAY_NL_REF = {
    11: 77.77, 21: 32.98, 22: 200.00, 31: 39.35, 32: 39.35,
    41: 107.80, 42: 142.68, 43: 107.80, 44: 107.80, 
    52: 78.23, 53: 200.00, 61: 298.70, 62: 300.00
}

TSYS_CODES = {"AUTO_C": "VOL_AUTO_C", "CA1_C": "VOL_CA1_C", "CA2_C": "VOL_CA2_C", "CA_BD": "VOL_CA_BD", "CU_C": "VOL_CU_C"}

# ==============================================================================
# 3. CONFIGURACIÓN TRANSPORTE PÚBLICO (TP)
# ==============================================================================

ITEMS_BASE_DIR_TP = Path(r"D:\VISUM\Archivos de integración MOVES_VISUM\InputData\20251002\Transit\Timeprofileitems")
FILE_PROFILES_TP = Path(r"C:\Users\Eydan\OneDrive\Escritorio\ITESM\MAITEC Lab\VISUM_MOVES Integration\NBs for MOVES VISUM\NB6_UIDB for TP\NB6_Input Data\extract\Timeprofiles\scenario=TransitAP\Timeprofiles.parquet")

HEADWAY_IS_SECONDS = True 
TRANSIT_BUS_ID = 42
ROAD_TYPE_SPLIT_TP = {4: 0.8336, 5: 0.1664}
ACTIVE_ROAD_TYPES_TP = [4, 5] 

FOLDER_MAPPING_TP = {
    "EM": "scenario=TransitEM", "AP": "scenario=TransitAP", "LM": "scenario=TransitLM",
    "MP": "scenario=TransitMP", "EA": "scenario=TransitEA", "PP": "scenario=TransitPP", "EV": "scenario=TransitEV"
}

HEADWAY_COLS = {
    "EM": "HEADWAYEM", "AP": "HEADWAYAP", "LM": "HEADWAYLM",
    "MP": "HEADWAYMP", "EA": "HEADWAYEA", "PP": "HEADWAYPP", "EV": "HEADWAYEV"
}

def clean_visum_string(val):
    if pd.isna(val) or val == "": return 0.0
    if isinstance(val, (int, float)): return float(val)
    val_str = str(val).lower()
    clean_str = re.sub(r'[^\d\.]', '', val_str)
    try: return float(clean_str)
    except ValueError: return 0.0

# ==============================================================================
# 4. PROCESAMIENTO DE AUTO Y CARGA (SÚPER VECTORIZADO)
# ==============================================================================
print("🚗 Iniciando procesamiento de Auto y Carga...")

all_dfs_auto = []
available_files_auto = list(INPUT_VISUM_DIR_AUTO.glob("*.parquet"))

for periodo, file_tag in FILES_MAPPING_AUTO.items():
    match = next((f for f in available_files_auto if file_tag in f.name), None)
    if match:
        df = pd.read_parquet(match)
        len_col = next((c for c in df.columns if c.upper() == 'LENGTH'), None)
        df['LENGTH_CLEAN'] = df[len_col].apply(clean_visum_string) if len_col else 0.0
        if 'TYPENO' not in df.columns: df['TYPENO'] = 0

        factor = VOLUME_FACTORS.get(periodo, 1.0)
        for tsys_code, std_name in TSYS_CODES.items():
            col_vol_found = next((c for c in df.columns if "VOL" in c.upper() and tsys_code in c.upper()), None)
            col_spd_found = next((c for c in df.columns if ("VCUR" in c.upper() or "SPD" in c.upper()) and tsys_code in c.upper()), None)
            
            if col_vol_found:
                cleaned_vals = df[col_vol_found].apply(clean_visum_string)
                df[std_name] = cleaned_vals / factor if factor not in [0, 1.0] else cleaned_vals
            else:
                df[std_name] = 0.0
            
            df[f"SPD_{tsys_code}"] = df[col_spd_found].apply(clean_visum_string) if col_spd_found else 0.0

        df['Periodo'] = periodo
        cols_to_keep = ['LENGTH_CLEAN', 'TYPENO', 'Periodo'] + list(TSYS_CODES.values()) + [f"SPD_{k}" for k in TSYS_CODES.keys()]
        df_final = df[[c for c in cols_to_keep if c in df.columns]].copy()
        df_final.rename(columns={'LENGTH_CLEAN': 'LENGTH'}, inplace=True)
        all_dfs_auto.append(df_final)

if not all_dfs_auto: raise ValueError("No se pudieron cargar datos de Auto.")
df_visum_master = pd.concat(all_dfs_auto, ignore_index=True)
df_visum_master['LinkID_idx'] = df_visum_master.index # ID único temporal para cruzar columnas

print("   ⚡ Aplicando optimización vectorial de memoria...")
# A. Extraer y filtrar volúmenes en formato largo instantáneamente
vol_cols = list(TSYS_CODES.values())
df_vol = df_visum_master.melt(id_vars=['LinkID_idx', 'Periodo', 'LENGTH', 'TYPENO'],
                              value_vars=vol_cols, var_name='VOL_COL', value_name='Volume')
df_vol = df_vol[df_vol['Volume'] > 0].copy() # 🗑️ El secreto de la memoria: descartar ceros inmediatamente

# B. Extraer velocidades
spd_cols = [f"SPD_{k}" for k in TSYS_CODES.keys()]
df_spd = df_visum_master.melt(id_vars=['LinkID_idx'],
                              value_vars=spd_cols, var_name='SPD_COL', value_name='Speed_kph')

# C. Cruzar Volumen con su Velocidad correspondiente
df_vol['SPD_COL'] = df_vol['VOL_COL'].str.replace('VOL_', 'SPD_')
df_vol = df_vol.merge(df_spd, on=['LinkID_idx', 'SPD_COL'], how='inner')

# Liberar RAM
del df_visum_master, df_spd
gc.collect()

# D. Expandir los tipos de vehículos según el mapeo
map_rows = []
total_auto = sum(AUTO_LIGHT_MAPPING.values())
for k, v in AUTO_LIGHT_MAPPING.items():
    map_rows.append({'VOL_COL': 'VOL_AUTO_C', 'sourceTypeID': k, 'share': v / total_auto})
map_rows.extend([
    {'VOL_COL': 'VOL_CA1_C', 'sourceTypeID': 61, 'share': 1.0},
    {'VOL_COL': 'VOL_CA2_C', 'sourceTypeID': 61, 'share': 1.0},
    {'VOL_COL': 'VOL_CA_BD', 'sourceTypeID': 61, 'share': 1.0},
    {'VOL_COL': 'VOL_CU_C',  'sourceTypeID': 52, 'share': 1.0}
])
df_map = pd.DataFrame(map_rows)

df_vol = df_vol.merge(df_map, on='VOL_COL', how='inner')

# Cálculos físicos base
df_vol['VKT'] = df_vol['Volume'] * df_vol['LENGTH'] * df_vol['share']
df_vol['VMT'] = df_vol['VKT'] * MILES_PER_KM
df_vol['roadTypeID'] = np.where(df_vol['TYPENO'].isin([0, 1, 2, 10, 20]), 5, 4).astype(np.int8)

# --- AUTO: sourceTypePopulation ---
vkt_sum = df_vol.groupby('sourceTypeID')['VKT'].sum().reset_index()
df_ref = pd.DataFrame(list(KM_DAY_NL_REF.items()), columns=['sourceTypeID', 'KM_Day_NL'])
df_fleet_auto = pd.merge(vkt_sum, df_ref, on='sourceTypeID', how='left')
df_fleet_auto['sourceTypePopulation'] = (df_fleet_auto['VKT'] / df_fleet_auto['KM_Day_NL']).fillna(0).round(0).astype(int)
df_fleet_auto.insert(0, "yearID", YEAR_ID)
fleet_auto = df_fleet_auto[['yearID', 'sourceTypeID', 'sourceTypePopulation']]

# --- AUTO: VMT ---
vmt_base = df_vol.groupby('sourceTypeID')['VMT'].sum().reset_index()
grid_dates = list(itertools.product([YEAR_ID], range(1, 13), DAY_IDS))
df_dates = pd.DataFrame(grid_dates, columns=['yearID', 'monthID', 'dayID'])
vmt_auto = vmt_base.merge(df_dates, how='cross')[["sourceTypeID", "yearID", "monthID", "dayID", "VMT"]]
vmt_auto['VMT'] = vmt_auto['VMT'].round(4)

# --- AUTO: roadTypeVMTFraction ---
vmt_by_road = df_vol.groupby(['sourceTypeID', 'roadTypeID'])['VMT'].sum().reset_index()
vmt_by_road['roadTypeVMTFraction'] = vmt_by_road['VMT'] / vmt_by_road.groupby('sourceTypeID')['VMT'].transform('sum')

unique_sources = vmt_by_road['sourceTypeID'].unique()
grid = pd.DataFrame(list(itertools.product(unique_sources, ALL_ROAD_TYPES)), columns=['sourceTypeID', 'roadTypeID'])
road_auto = pd.merge(grid, vmt_by_road[['sourceTypeID', 'roadTypeID', 'roadTypeVMTFraction']], on=['sourceTypeID', 'roadTypeID'], how='left').fillna(0)

# --- AUTO: Configuración Horaria y Velocidad ---
hours_df = pd.DataFrame([(p, h) for p, hours in HOURS_MAPPING.items() for h in hours], columns=['Periodo', 'Hour'])
period_lengths = {p: len(h) for p, h in HOURS_MAPPING.items()}
df_vol['Period_Hours'] = df_vol['Periodo'].map(period_lengths)

df_vol['Speed_mph'] = df_vol['Speed_kph'].clip(lower=1.6) * MILES_PER_KM
df_vol['BIN_ID'] = np.digitize(df_vol['Speed_mph'], MOVES_SPEED_BINS).clip(max=16)
df_vol['Time_per_hour'] = (df_vol['VMT'] / df_vol['Speed_mph']) / df_vol['Period_Hours']
df_vol['VMT_per_hour'] = df_vol['VMT'] / df_vol['Period_Hours']

# --- AUTO: hourVMTFraction ---
df_hour_merge = df_vol[['sourceTypeID', 'roadTypeID', 'Periodo', 'VMT_per_hour']].merge(hours_df, on='Periodo')
df_hour_merge['hourID'] = df_hour_merge['Hour'] + 1
vmt_hour = df_hour_merge.groupby(['sourceTypeID', 'roadTypeID', 'hourID'])['VMT_per_hour'].sum().reset_index()
vmt_hour['hourVMTFraction'] = vmt_hour['VMT_per_hour'] / vmt_hour.groupby(['sourceTypeID', 'roadTypeID'])['VMT_per_hour'].transform('sum')

final_hours = []
for day in DAY_IDS:
    d = vmt_hour.copy()
    d['dayID'] = day
    final_hours.append(d)
hour_auto = pd.concat(final_hours)[['sourceTypeID', 'roadTypeID', 'dayID', 'hourID', 'hourVMTFraction']]
del df_hour_merge; gc.collect()

# --- AUTO: avgSpeedFraction ---
df_speed_merge = df_vol[['sourceTypeID', 'roadTypeID', 'Periodo', 'BIN_ID', 'Time_per_hour']].merge(hours_df, on='Periodo')
df_speed_merge['hourID'] = df_speed_merge['Hour'] + 1
grp_speed = df_speed_merge.groupby(['sourceTypeID', 'roadTypeID', 'hourID', 'BIN_ID'])['Time_per_hour'].sum().reset_index()
grp_speed.rename(columns={'BIN_ID': 'avgSpeedBinID'}, inplace=True)

# Llenar huecos de speed bins con ceros
unique_base = grp_speed[['sourceTypeID', 'roadTypeID', 'hourID']].drop_duplicates()
grid_full = unique_base.merge(pd.DataFrame({'avgSpeedBinID': ALL_BINS}), how='cross')
merged_speed = pd.merge(grid_full, grp_speed, on=['sourceTypeID', 'roadTypeID', 'hourID', 'avgSpeedBinID'], how='left').fillna(0)
merged_speed['avgSpeedFraction'] = merged_speed['Time_per_hour'] / merged_speed.groupby(['sourceTypeID', 'roadTypeID', 'hourID'])['Time_per_hour'].transform('sum')
merged_speed['avgSpeedFraction'] = merged_speed['avgSpeedFraction'].fillna(0)

final_rows = []
for day in DAY_IDS:
    d = merged_speed.copy()
    d['hourDayID'] = (d['hourID'].astype(str) + str(day)).astype(np.int64)
    final_rows.append(d)
speed_auto = pd.concat(final_rows)[['sourceTypeID', 'roadTypeID', 'hourDayID', 'avgSpeedBinID', 'avgSpeedFraction']]

del df_vol, df_speed_merge; gc.collect()

# ==============================================================================
# 5. PROCESAMIENTO TRANSPORTE PÚBLICO (TP)
# ==============================================================================
print("\n🚌 Iniciando procesamiento de Transporte Público...")

try: df_profiles = pd.read_parquet(FILE_PROFILES_TP)
except: df_profiles = pd.read_csv(FILE_PROFILES_TP)
df_profiles.columns = [c.strip().upper() for c in df_profiles.columns]

all_period_data_tp = []
for period_code, folder_name in FOLDER_MAPPING_TP.items():
    current_path = ITEMS_BASE_DIR_TP / folder_name
    found_file = list(current_path.glob("*.parquet"))
    
    if not found_file: 
        print(f"   ⚠️ SALTANDO {period_code}: No se encontró .parquet en {current_path}")
        continue
    
    print(f"   -> Procesando [{period_code}]: {found_file[0].name}")
    df_items = pd.read_parquet(found_file[0])
    df_items.columns = [c.strip().upper() for c in df_items.columns]
    
    col_len = 'POSTLENGTH' if 'POSTLENGTH' in df_items.columns else 'LENGTH'
    df_items['LENGTH_KM'] = df_items[col_len].apply(clean_visum_string)
    df_items['SPEED_KPH'] = df_items['VEL'].apply(clean_visum_string) if 'VEL' in df_items.columns else 0.0
        
    merge_keys = [c for c in ['LINENAME', 'LINEROUTENAME', 'DIRECTIONCODE', 'LINEROUTEID'] if c in df_items.columns and c in df_profiles.columns]
    df_merged = pd.merge(df_items, df_profiles, on=merge_keys, how='left')
    
    hw_col = HEADWAY_COLS.get(period_code)
    if hw_col and hw_col in df_merged.columns:
        headway_vals = df_merged[hw_col].apply(clean_visum_string) * (1.0 if HEADWAY_IS_SECONDS else 60.0)
        freq_per_hour = np.where(headway_vals > 0, 3600 / headway_vals, 0)
        df_merged['VOL_PERIOD'] = freq_per_hour * len(HOURS_MAPPING[period_code])
    else:
        df_merged['VOL_PERIOD'] = 0.0

    df_merged['PERIOD_CODE'] = period_code
    all_period_data_tp.append(df_merged[['PERIOD_CODE', 'LENGTH_KM', 'SPEED_KPH', 'VOL_PERIOD']].copy())

if not all_period_data_tp: 
    raise ValueError(f"❌ No se generaron datos de TP. Ninguna de las carpetas en {ITEMS_BASE_DIR_TP} tenía archivos .parquet")

df_master_bus = pd.concat(all_period_data_tp, ignore_index=True)

# --- TP: sourceTypePopulation & VMT ---
total_vmt_miles = (df_master_bus['VOL_PERIOD'] * df_master_bus['LENGTH_KM'] * MILES_PER_KM).sum()
pop_bus = int(round((total_vmt_miles / MILES_PER_KM) / KM_DAY_NL_REF[42])) if KM_DAY_NL_REF[42] > 0 else 0

fleet_tp = pd.DataFrame([{'yearID': YEAR_ID, 'sourceTypeID': TRANSIT_BUS_ID, 'sourceTypePopulation': pop_bus}])

grid_dates_tp = list(itertools.product([YEAR_ID], range(1, 13), DAY_IDS))
df_vmt_tp = pd.DataFrame(grid_dates_tp, columns=['yearID', 'monthID', 'dayID'])
df_vmt_tp['sourceTypeID'] = TRANSIT_BUS_ID
df_vmt_tp['VMT'] = total_vmt_miles
vmt_tp = df_vmt_tp[["sourceTypeID", "yearID", "monthID", "dayID", "VMT"]].round(4)

# --- TP: roadTypeVMTFraction ---
grid_rt = pd.DataFrame(list(itertools.product([TRANSIT_BUS_ID], ALL_ROAD_TYPES)), columns=['sourceTypeID', 'roadTypeID'])
grid_rt['roadTypeVMTFraction'] = grid_rt['roadTypeID'].map(ROAD_TYPE_SPLIT_TP).fillna(0.0)
total_frac = grid_rt['roadTypeVMTFraction'].sum()
if not np.isclose(total_frac, 1.0): grid_rt['roadTypeVMTFraction'] = grid_rt['roadTypeVMTFraction'] / total_frac
road_tp = grid_rt

# --- TP: avgSpeedFraction ---
speed_base_records = []
df_active = df_master_bus[df_master_bus['VOL_PERIOD'] > 0].copy()
if not df_active.empty:
    df_active['SPEED_MPH'] = df_active['SPEED_KPH'] * MILES_PER_KM
    df_active['SPEED_MPH'] = df_active['SPEED_MPH'].replace(0, 1.0)
    df_active['DIST_MILES'] = df_active['LENGTH_KM'] * MILES_PER_KM
    df_active['TOTAL_TIME_H'] = df_active['VOL_PERIOD'] * (df_active['DIST_MILES'] / df_active['SPEED_MPH'])
    df_active['BIN_ID'] = np.digitize(df_active['SPEED_MPH'], MOVES_SPEED_BINS)
    df_active.loc[df_active['BIN_ID'] > 16, 'BIN_ID'] = 16
    grouped_speed = df_active.groupby(['PERIOD_CODE', 'BIN_ID'])['TOTAL_TIME_H'].sum()

    for (period, bin_id), total_time in grouped_speed.items():
        hours_list = HOURS_MAPPING.get(period, [])
        if not hours_list: continue
        time_per_hour = total_time / len(hours_list)
        for h in hours_list:
            speed_base_records.append({'hourID': h + 1, 'avgSpeedBinID': bin_id, 'time_weight': time_per_hour})

df_base = pd.DataFrame(speed_base_records)
if not df_base.empty:
    df_base['avgSpeedFraction'] = df_base['time_weight'] / df_base.groupby('hourID')['time_weight'].transform('sum')
else:
    df_base = pd.DataFrame(columns=['hourID', 'avgSpeedBinID', 'avgSpeedFraction'])

df_grid = pd.DataFrame(list(itertools.product([TRANSIT_BUS_ID], ALL_ROAD_TYPES, ALL_HOURS, ALL_BINS)), columns=['sourceTypeID', 'roadTypeID', 'hourID', 'avgSpeedBinID'])
df_merged = pd.merge(df_grid, df_base[['hourID', 'avgSpeedBinID', 'avgSpeedFraction']], on=['hourID', 'avgSpeedBinID'], how='left')
df_merged['avgSpeedFraction'] = df_merged['avgSpeedFraction'].fillna(0)

df_merged.loc[~df_merged['roadTypeID'].isin(ACTIVE_ROAD_TYPES_TP), 'avgSpeedFraction'] = 0
sums = df_merged.groupby(['sourceTypeID', 'roadTypeID', 'hourID'])['avgSpeedFraction'].transform('sum')
df_merged.loc[(sums == 0) & (df_merged['avgSpeedBinID'] == 1), 'avgSpeedFraction'] = 1.0

final_rows = []
for day in DAY_IDS:
    d = df_merged.copy()
    d['hourDayID'] = (d['hourID'].astype(str) + str(day)).astype(np.int64)
    final_rows.append(d)
speed_tp = pd.concat(final_rows)[['sourceTypeID', 'roadTypeID', 'hourDayID', 'avgSpeedBinID', 'avgSpeedFraction']]

# --- TP: hourVMTFraction ---
df_master_bus['VMT_SEGMENT'] = df_master_bus['VOL_PERIOD'] * df_master_bus['LENGTH_KM'] * MILES_PER_KM
vmt_by_period = df_master_bus.groupby('PERIOD_CODE')['VMT_SEGMENT'].sum()
hourly_vmt_base = {h: 0.0 for h in ALL_HOURS}
for period, vmt_total in vmt_by_period.items():
    hours_list = HOURS_MAPPING.get(period, [])
    if not hours_list: continue
    for h in hours_list: hourly_vmt_base[h + 1] = vmt_total / len(hours_list) 

total_daily_vmt = sum(hourly_vmt_base.values())
hourly_profile = {h: vmt / total_daily_vmt for h, vmt in hourly_vmt_base.items()} if total_daily_vmt > 0 else {h: 1.0 / 24.0 for h in ALL_HOURS}

grid_h_records = []
for rt in ALL_ROAD_TYPES:
    for h in ALL_HOURS:
        grid_h_records.append({
            'sourceTypeID': TRANSIT_BUS_ID, 'roadTypeID': rt, 'hourID': h,
            'hourVMTFraction': hourly_profile.get(h, 0.0) if rt in ACTIVE_ROAD_TYPES_TP else (1.0 / 24.0)
        })

final_h_list = []
df_hour = pd.DataFrame(grid_h_records)
for day in DAY_IDS:
    d = df_hour.copy()
    d['dayID'] = day
    final_h_list.append(d)
hour_tp = pd.concat(final_h_list)[['sourceTypeID', 'roadTypeID', 'dayID', 'hourID', 'hourVMTFraction']]

# ==============================================================================
# 6. UNIFICACIÓN Y EXPORTACIÓN FINAL
# ==============================================================================
print("\n🔗 Unificando y exportando tablas...")

final_fleet = pd.concat([fleet_auto, fleet_tp], ignore_index=True)
final_vmt   = pd.concat([vmt_auto, vmt_tp], ignore_index=True)
final_road  = pd.concat([road_auto, road_tp], ignore_index=True)
final_speed = pd.concat([speed_auto, speed_tp], ignore_index=True)
final_hour  = pd.concat([hour_auto, hour_tp], ignore_index=True)

final_fleet.to_csv(OUTPUT_DIR / "sourceTypePopulation.csv", index=False)
final_vmt.to_csv(OUTPUT_DIR / "VMT.csv", index=False)
final_road.to_csv(OUTPUT_DIR / "roadTypeVMTFraction.csv", index=False)
final_speed.to_csv(OUTPUT_DIR / "avgSpeedFraction.csv", index=False)
final_hour.to_csv(OUTPUT_DIR / "hourVMTFraction.csv", index=False)

print(f"✅ Proceso completo. Todas las tablas unificadas se guardaron en:\n{OUTPUT_DIR}")

🚗 Iniciando procesamiento de Auto y Carga...
   ⚡ Aplicando optimización vectorial de memoria...

🚌 Iniciando procesamiento de Transporte Público...
   ⚠️ SALTANDO EM: No se encontró .parquet en D:\VISUM\Archivos de integración MOVES_VISUM\InputData\20251002\Transit\Timeprofileitems\scenario=TransitEM
   -> Procesando [AP]: Timeprofileitems.parquet
   -> Procesando [LM]: Timeprofileitems.parquet
   -> Procesando [MP]: Timeprofileitems.parquet
   -> Procesando [EA]: Timeprofileitems.parquet
   -> Procesando [PP]: Timeprofileitems.parquet
   -> Procesando [EV]: Timeprofileitems.parquet

🔗 Unificando y exportando tablas...
✅ Proceso completo. Todas las tablas unificadas se guardaron en:
C:\Users\Eydan\OneDrive\Escritorio\ITESM\MAITEC Lab\VISUM_MOVES Integration\NBs for MOVES VISUM\NB2_MOVES UIDB from VISUM\NB2_OutputData\MOVES_UIDB_Tables_VISUM\Complete Tables
